<a href="https://colab.research.google.com/github/seandaza/Python-The-Fundamentals/blob/master/Dash.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# DASH

> Dash es una biblioteca de Python de código abierto para crear aplicaciones reactivas basadas en la web. Permite el análisis de datos, exploración de datos, visualización, modelado, control de instrumentos e informes.

> El código de la aplicación Dash es declarativo y reactivo, lo que facilita la creación de aplicaciones complejas que contienen muchos elementos interactivos





> Esencialmente, las aplicaciones Dash se componen de dos partes: (1) el "diseño" de la aplicación que describe la apariencia de la aplicación, y (2) las "devoluciones de llamada" que permiten que las aplicaciones sean interactivas. Veamos un ejemplo simple a continuación:



##Instalemos Dependencias

In [ ]:
!pip install jupyter-dash -q
!pip install pyngrok
!pip install jupyterlab_dash
!pip install dash

In [ ]:
from jupyter_dash import JupyterDash  # pip install dash
from pyngrok import ngrok
import dash_html_components as html
import dash_core_components as dcc
from dash.dependencies import Output, Input
import pandas as pd  # pip install pandas
import plotly.express as px
import math
import dash
from dash import no_update

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: UserWarning: 
The dash_html_components package is deprecated. Please replace
`import dash_html_components as html` with `from dash import html`
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: UserWarning: 
The dash_core_components package is deprecated. Please replace
`import dash_core_components as dcc` with `from dash import dcc`
  after removing the cwd from sys.path.


##*Graficos*

> La biblioteca `dash_core_components` incluye un componente llamado `Graph`.

> Graph presenta visualizaciones de datos interactivas utilizando la biblioteca de gráficos de JavaScript `plotly.js` de código abierto. `Plotly.js` admite más de 35 tipos de gráficos y los renderiza tanto en SVG de calidad vectorial como en WebGL de alto rendimiento. Catacteristicas:

> 1. El diseño se compone de un árbol de "componentes" como `html.Div` y `dcc.Graph`.

> 2. La función `dash_html_components` (`dash.html` a partir de Dash v2.0) tiene un componente para cada etiqueta HTML. El componente `html.H1` (children = 'Hello Dash') genera un elemento HTML `<h1> Hello Dash </h1>` en su aplicación

> 3. No todos los componenetes son `HTML` puro. los `dash_core_components` describen un alto nivel de componentes que son interactivos y son generados con Javascript, HTML y CSS a traves de la librería de React.js.

> 4. Cada componente es descrito a través de palabras que describen atributos de manera declarativa

> 5. La propiedad `Children` es especial, por convención, siempre es el primer atributo, lo que significa que puede omitirlo: `html.H1 (children = 'Hello Dash')` es lo mismo que `html.H1 ('Hello Dash')`. Puede contener una cadena, un número, un solo componente o una lista de componentes

##Grafico de Barras

> Con un dataframe local, contruyamos graficos de barras


In [ ]:
external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']
app = JupyterDash(__name__, external_stylesheets=external_stylesheets)

#Datos
df = pd.DataFrame({
    "Fruit": ["Apples", "Oranges", "Bananas", "Apples", "Oranges", "Bananas"],
    "Amount": [4, 1, 2, 2, 4, 5],
    "City": ["SF", "SF", "SF", "Montreal", "Montreal", "Montreal"]
})

fig = px.bar(df, x="Fruit", y="Amount", color="City", barmode="group")

app.layout = html.Div(children=[
    html.H1(children='Hello Dash'),

    html.Div(children='''
        Dash: A web application framework for your data.
    '''),

    dcc.Graph(
        id='example-graph',
        figure=fig
    )
])

app.run_server(mode='inline', port='1233')

##ScatterPlot

> Aquí hay un ejemplo que crea un diagrama de dispersión a partir de un marco de datos de Pandas tomado de un repositorio de datos externo que muestra la expectativa de vida por continente.

In [ ]:
%%writefile scatter.py
import dash
import pandas as pd
import plotly.express as px
import dash_html_components as html
import dash_core_components as dcc

app = dash.Dash(__name__)

df = pd.read_csv('https://gist.githubusercontent.com/chriddyp/5d1ea79569ed194d432e56108a04d188/raw/a9f9e8076b837d541398e999dcbac2b2826a81f8/gdp-life-exp-2007.csv')

fig = px.scatter(df, x="gdp per capita", y="life expectancy",
                 size="population", color="continent", hover_name="country",
                 log_x=True, size_max=60)

app.layout = html.Div([
    dcc.Graph(
        id='life-exp-vs-gdp',
        figure=fig
    )
])
if __name__ == '__main__':
    app.run_server(port='1234')

Una vez creado el archivo, abra un tunel con `ngrok` en el mismo puerto que sirve la aplicacion, ejecute el archivo, y haga click sobre el link que sirve `ngrok`

In [ ]:
ngrok.connect(1234)

Ejecute el programa:

In [ ]:
!python scatter.py

## Tablas

> A partir de un conjunto de datos de exportaciones de agricultura, podemos contruir tablas a través de la librería de Pandas.

In [ ]:
%%writefile table.py
import dash
import pandas as pd
from dash import html

app = dash.Dash(__name__)

df = pd.read_csv('https://gist.githubusercontent.com/chriddyp/c78bf172206ce24f77d6363a2d754b59/raw/c353e8ef842413cae56ae3920b8fd78468aa4cb2/usa-agricultural-exports-2011.csv')


def generate_table(dataframe, max_rows=10):
    return html.Table([
        html.Thead(
            html.Tr([html.Th(col) for col in dataframe.columns])
        ),
        html.Tbody([
            html.Tr([
                html.Td(dataframe.iloc[i][col]) for col in dataframe.columns
            ]) for i in range(min(len(dataframe), max_rows))
        ])
    ])




app.layout = html.Div([
    html.H4(children='US Agriculture Exports (2011)'),
    generate_table(df)
])

if __name__ == '__main__':
    app.run_server(port='1235')

In [ ]:
ngrok.connect(1235)

In [ ]:
!python table.py



---



#@callbacks

> Tambien se pueden crear aplicaciones Dash usando funciones de devolución de llamada `callback functions`: funciones que son llamadas automáticamente por Dash cada vez que cambia la propiedad de un componente de entrada, para actualizar alguna propiedad en otro componente (la salida).

> Para un rendimiento óptimo de la interacción del usuario y la carga de gráficos, las aplicaciones de producción de Dash deben considerar las capacidades de cola de trabajos, HPC, Datashader y escalamiento horizontal de Dash Enterprise.

> Comencemos con un ejemplo simple de una aplicación Dash interactiva.



##Bars

In [ ]:
%%writefile bars.py
import dash
from dash import dcc
from dash import html
from dash.dependencies import Input, Output
import plotly.express as px

df = px.data.tips()
days = df.day.unique()

app = dash.Dash(__name__)

app.layout = html.Div([
    dcc.Dropdown(
        id="dropdown",
        options=[{"label": x, "value": x} for x in days],
        value=days[0],
        clearable=False,
    ),
    dcc.Graph(id="bar-chart"),
])

@app.callback(
    Output("bar-chart", "figure"), 
    [Input("dropdown", "value")])
def update_bar_chart(day):
    mask = df["day"] == day
    fig = px.bar(df[mask], x="sex", y="total_bill", 
                 color="smoker", barmode="group")
    return fig

app.run_server(port='2030')

In [ ]:
ngrok.connect(2030)

In [ ]:
!python bars.py

##Hovering

In [ ]:
%%writefile hovering.py
import dash
from dash import dcc
from dash import html
import plotly.graph_objs as go
import pandas as pd

app = dash.Dash()

df = pd.read_csv(
    'https://raw.githubusercontent.com/'
    'plotly/datasets/master/'
    '1962_2006_walmart_store_openings.csv')

app.layout = html.Div([
    html.H1('Walmart Store Openings'),
    html.Div(id='text-content'),
    dcc.Graph(id='map', figure={
        'data': [{
            'lat': df['LAT'],
            'lon': df['LON'],
            'marker': {
                'color': df['YEAR'],
                'size': 8,
                'opacity': 0.6
            },
            'customdata': df['storenum'],
            'type': 'scattermapbox'
        }],
        'layout': {
            'mapbox': {
                'accesstoken': 'pk.eyJ1IjoiY2hyaWRkeXAiLCJhIjoiY2ozcGI1MTZ3MDBpcTJ3cXR4b3owdDQwaCJ9.8jpMunbKjdq1anXwU5gxIw'
            },
            'hovermode': 'closest',
            'margin': {'l': 0, 'r': 0, 'b': 0, 't': 0}
        }
    })
])

@app.callback(
    dash.dependencies.Output('text-content', 'children'),
    [dash.dependencies.Input('map', 'hoverData')])
def update_text(hoverData):
    s = df[df['storenum'] == hoverData['points'][0]['customdata']]
    return html.H3(
        'The {}, {} {} opened in {}'.format(
            s.iloc[0]['STRCITY'],
            s.iloc[0]['STRSTATE'],
            s.iloc[0]['type_store'],
            s.iloc[0]['YEAR']
        )
    )

app.css.append_css({
    'external_url': 'https://codepen.io/chriddyp/pen/bWLwgP.css'
})

if __name__ == '__main__':
    app.run_server(port='1236')

In [ ]:
ngrok.connect(1236)

In [ ]:
!python hovering.py

## CrossFilter

In [ ]:
%%writefile crossfilter.py
import dash
from dash import dcc
from dash import html
import plotly.graph_objs as go
import pandas as pd

app = dash.Dash()

df = pd.read_csv(
    'https://gist.githubusercontent.com/chriddyp/'
    'cb5392c35661370d95f300086accea51/raw/'
    '8e0768211f6b747c0db42a9ce9a0937dafcbd8b2/'
    'indicators.csv')

available_indicators = df['Indicator Name'].unique()

app.layout = html.Div([
    html.Div([

        html.Div([
            dcc.Dropdown(
                id='crossfilter-xaxis-column',
                options=[{'label': i, 'value': i} for i in available_indicators],
                value='Fertility rate, total (births per woman)'
            ),
            dcc.RadioItems(
                id='crossfilter-xaxis-type',
                options=[{'label': i, 'value': i} for i in ['Linear', 'Log']],
                value='Linear',
                labelStyle={'display': 'inline-block'}
            )
        ],
        style={'width': '49%', 'display': 'inline-block'}),

        html.Div([
            dcc.Dropdown(
                id='crossfilter-yaxis-column',
                options=[{'label': i, 'value': i} for i in available_indicators],
                value='Life expectancy at birth, total (years)'
            ),
            dcc.RadioItems(
                id='crossfilter-yaxis-type',
                options=[{'label': i, 'value': i} for i in ['Linear', 'Log']],
                value='Linear',
                labelStyle={'display': 'inline-block'}
            )
        ], style={'width': '49%', 'float': 'right', 'display': 'inline-block'})
    ], style={
        'borderBottom': 'thin lightgrey solid',
        'backgroundColor': 'rgb(250, 250, 250)',
        'padding': '10px 5px'
    }),

    html.Div([
        dcc.Graph(
            id='crossfilter-indicator-scatter',
            hoverData={'points': [{'customdata': 'Japan'}]}
        )
    ], style={'width': '49%', 'display': 'inline-block', 'padding': '0 20'}),
    html.Div([
        dcc.Graph(id='x-time-series'),
        dcc.Graph(id='y-time-series'),
    ], style={'display': 'inline-block', 'width': '49%'}),

    html.Div(dcc.Slider(
        id='crossfilter-year--slider',
        min=df['Year'].min(),
        max=df['Year'].max(),
        value=df['Year'].max(),
        step=None,
        marks={str(year): str(year) for year in df['Year'].unique()}
    ), style={'width': '49%', 'padding': '0px 20px 20px 20px'})
])

@app.callback(
    dash.dependencies.Output('crossfilter-indicator-scatter', 'figure'),
    [dash.dependencies.Input('crossfilter-xaxis-column', 'value'),
     dash.dependencies.Input('crossfilter-yaxis-column', 'value'),
     dash.dependencies.Input('crossfilter-xaxis-type', 'value'),
     dash.dependencies.Input('crossfilter-yaxis-type', 'value'),
     dash.dependencies.Input('crossfilter-year--slider', 'value')])
def update_graph(xaxis_column_name, yaxis_column_name,
                 xaxis_type, yaxis_type,
                 year_value):
    dff = df[df['Year'] == year_value]

    return {
        'data': [go.Scatter(
            x=dff[dff['Indicator Name'] == xaxis_column_name]['Value'],
            y=dff[dff['Indicator Name'] == yaxis_column_name]['Value'],
            text=dff[dff['Indicator Name'] == yaxis_column_name]['Country Name'],
            customdata=dff[dff['Indicator Name'] == yaxis_column_name]['Country Name'],
            mode='markers',
            marker={
                'size': 15,
                'opacity': 0.5,
                'line': {'width': 0.5, 'color': 'white'}
            }
        )],
        'layout': go.Layout(
            xaxis={
                'title': xaxis_column_name,
                'type': 'linear' if xaxis_type == 'Linear' else 'log'
            },
            yaxis={
                'title': yaxis_column_name,
                'type': 'linear' if yaxis_type == 'Linear' else 'log'
            },
            margin={'l': 40, 'b': 30, 't': 10, 'r': 0},
            height=450,
            hovermode='closest'
        )
    }

def create_time_series(dff, axis_type, title):
    return {
        'data': [go.Scatter(
            x=dff['Year'],
            y=dff['Value'],
            mode='lines+markers'
        )],
        'layout': {
            'height': 225,
            'margin': {'l': 20, 'b': 30, 'r': 10, 't': 10},
            'annotations': [{
                'x': 0, 'y': 0.85, 'xanchor': 'left', 'yanchor': 'bottom',
                'xref': 'paper', 'yref': 'paper', 'showarrow': False,
                'align': 'left', 'bgcolor': 'rgba(255, 255, 255, 0.5)',
                'text': title
            }],
            'yaxis': {'type': 'linear' if axis_type == 'Linear' else 'log'},
            'xaxis': {'showgrid': False}
        }
    }

@app.callback(
    dash.dependencies.Output('x-time-series', 'figure'),
    [dash.dependencies.Input('crossfilter-indicator-scatter', 'hoverData'),
     dash.dependencies.Input('crossfilter-xaxis-column', 'value'),
     dash.dependencies.Input('crossfilter-xaxis-type', 'value')])
def update_y_timeseries(hoverData, xaxis_column_name, axis_type):
    country_name = hoverData['points'][0]['customdata']
    dff = df[df['Country Name'] == country_name]
    dff = dff[dff['Indicator Name'] == xaxis_column_name]
    title = '<b>{}</b><br>{}'.format(country_name, xaxis_column_name)
    return create_time_series(dff, axis_type, title)

@app.callback(
    dash.dependencies.Output('y-time-series', 'figure'),
    [dash.dependencies.Input('crossfilter-indicator-scatter', 'hoverData'),
     dash.dependencies.Input('crossfilter-yaxis-column', 'value'),
     dash.dependencies.Input('crossfilter-yaxis-type', 'value')])
def update_x_timeseries(hoverData, yaxis_column_name, axis_type):
    dff = df[df['Country Name'] == hoverData['points'][0]['customdata']]
    dff = dff[dff['Indicator Name'] == yaxis_column_name]
    return create_time_series(dff, axis_type, yaxis_column_name)

app.css.append_css({
    'external_url': 'https://codepen.io/chriddyp/pen/bWLwgP.css'
})

if __name__ == '__main__':
    app.run_server()

In [ ]:
ngrok.connect(8050)

In [ ]:
!python crossfilter.py



---



#Plotly Extension

> El elemento `Graph` de Dash comparte la misma sintaxis que la biblioteca `plotly.py` de código abierto, por lo que puede cambiar fácilmente entre los dos. El componente `Graph` de `Dash` se conecta al sistema de eventos `plotly.js`, lo que permite a Dash escribir aplicaciones que respondan al pasar el mouse, hacer clic o seleccionar puntos en un gráfico de Plotly.



## Pie Chart

In [ ]:
import plotly.express as px

#datos tomados de https://www.gapminder.org/data/
df = px.data.gapminder().query("year == 2007").query("continent == 'Americas'")
fig = px.pie(df, values='pop', names='country',
             title='Population of American continent',
             hover_data=['lifeExp'], labels={'lifeExp':'life expectancy'})
fig.update_traces(textposition='inside', textinfo='percent+label')
fig.show()

### Pulling Sectors

> Para un diseño "extraído" o "ampliado" del gráfico circular, utilice el argumento `pull`. Puede ser un escalar para extraer todos los sectores o una matriz para extraer solo algunos de los sectores.



In [ ]:
import plotly.graph_objects as go

labels = ['Oxygen','Hydrogen','Carbon_Dioxide','Nitrogen']
values = [4500, 2500, 1053, 500]

# pull is given as a fraction of the pie radius
fig = go.Figure(data=[go.Pie(labels=labels, values=values, pull=[0, 0, 0.2, 0])])
fig.show()

## Donut Chart

In [ ]:
import plotly.graph_objects as go

labels = ['Oxygen','Hydrogen','Carbon_Dioxide','Nitrogen']
values = [4500, 2500, 1053, 500]

# Use `hole` to create a donut-like pie chart
fig = go.Figure(data=[go.Pie(labels=labels, values=values, hole=.3)])
fig.show()

## Bar Chart

In [ ]:
import plotly.express as px

#datos tomados de https://www.gapminder.org/data/
data_canada = px.data.gapminder().query("country == 'Canada'")
fig = px.bar(data_canada, x='year', y='pop')
fig.show()

## Bubble Chart

In [ ]:
import plotly.express as px

df = px.data.gapminder()

#datos tomados de https://www.gapminder.org/data/
fig = px.scatter(df.query("year==2007"), x="gdpPercap", y="lifeExp",
	         size="pop", color="continent",
                 hover_name="country", log_x=True, size_max=60)
fig.show()

## Heat Maps

In [ ]:
import plotly.express as px

#Data tomada de https://www.kaggle.com/sanjanabasu/tips-dataset
df = px.data.tips()

fig = px.density_heatmap(df, x="total_bill", y="tip", facet_row="sex", facet_col="smoker")
fig.show()



---



---



# Componentes Dash

## DropDown

In [ ]:
%%writefile dropdown.py
import dash
from dash import dcc
from dash import html

app = dash.Dash(__name__)

app.layout = html.Div([
    dcc.Dropdown(
        options=[
            {'label': 'New York City', 'value': 'NYC'},
            {'label': 'Montréal', 'value': 'MTL'},
            {'label': 'San Francisco', 'value': 'SF'}
        ],
        value='MTL'
    )
])

if __name__ == '__main__':
    app.run_server(debug=True)

Overwriting dropdown.py


In [ ]:
ngrok.connect(8050)

In [ ]:
!python dropdown.py

##Sliders

In [ ]:
%%writefile slider.py
import dash
from dash import dcc
from dash import html

app = dash.Dash(__name__)

app.layout = html.Div([
    dcc.Slider(
        min=-5,
        max=10,
        step=0.5,
        value=-3
    )
])

if __name__ == '__main__':
    app.run_server(port='2538')

In [ ]:
ngrok.connect(2538)

In [ ]:
!python slider.py

##Inputs

In [ ]:
%%writefile input.py
import dash
from dash import dcc
from dash import html

app = dash.Dash(__name__)

app.layout = html.Div([
    dcc.Input(
        placeholder='Enter a value...',
        type='text',
        value=''
    )
])

if __name__ == '__main__':
    app.run_server(port='2539')

In [ ]:
ngrok.connect(2539)

In [ ]:
!python input.py

##Checkboxes

In [ ]:
%%writefile checkbox.py
import dash
from dash import dcc
from dash import html

app = dash.Dash(__name__)

app.layout = html.Div([
    dcc.Checklist(
        options=[
            {'label': 'New York City', 'value': 'NYC'},
            {'label': 'Montréal', 'value': 'MTL'},
            {'label': 'San Francisco', 'value': 'SF'}
        ],
        value=['MTL', 'SF']
    )
])

if __name__ == '__main__':
    app.run_server(port='2540')

In [ ]:
ngrok.connect(2540)

In [ ]:
!python checkbox.py

##Markdown

In [ ]:
%%writefile markdown.py
import dash
from dash import dcc
from dash import html

app = dash.Dash(__name__)

app.layout = html.Div([
    dcc.Markdown('''
        #### Dash and Markdown
        Dash supports [Markdown](http://commonmark.org/help).
        Markdown is a simple way to write and format text.
        It includes a syntax for things like **bold text** and *italics*,
        [links](http://commonmark.org/help), inline `code` snippets, lists,
        quotes, and more.
    ''')
])

if __name__ == '__main__':
    app.run_server(port='2551')

Overwriting markdown.py


In [ ]:
ngrok.connect(2551)

In [ ]:
!python markdown.py